# Homework 2  Entity Resolution
# Baran Akyol
# e-mail: ba2500@columbia.edu | username: Baran

In [6]:
import pandas as pd
from operator import itemgetter
import time
import numpy as np
import re
from difflib import SequenceMatcher

I started off with some exploratory data analysis and looked at the datasets, the data types used, how noisy the data was, which features were included for which set and than did a quick comparison of the two. After that I set about writing a simple baseline algorithm. The first step was extracting data from the .csv files and processing it so that dtypes and the units were same for both. I converted every time to minutes, inserted the null value of 99999 for corrupt data and extracted the director name and stars' names as str, one column for director and one column for stars.

The algorithm works like a pipeline, starts with some rigid rules and eventualy relaxes those rules to come up with the best possible answer instead of a certainly correct answer. First I do a very simple comparison of time, director and stars names. If all these columns match, as there is no way of getting any more information, the algorithm returns 1, if not it passes on the next decision rule. The next relaxes the constraints and does not check for a complete match but instead leaves a margin. In the case of multiple matches, it returns the highest number of True values returned for stars' names. Since Rotten_tomatoes csv seems to contain more information than the Amazon one, I chose to check if the values of Amazon were within the rotten_tomatoes values and not the other way around.

My first baseline algorithm compared the given id with every other id in the rotten_tomatoes.csv and returned the most probable result and checked if the result was equal to the result given in the test set. This took too long however and wasn't scalable. In order to make the algorithm become feasible, I only use the two rows given and at most select from a small list of potential matches. This results in some information loss however but can be compensated for by tuning the parameters that set the margins.

There are two functions defined extractor_from_amazon and extractor_from_rotten which scrape the csv files and are used by make_new_df  function to create new pandas dataframe with clean data. compare_id function makes a binary comparison and  compare_multiple_ids does comparison in a for loop for every item in a given set. My final f score on the training set for the baseline was 76.36 with precision being  0.78 and recall being: 0.75 

My second attempt uses the same framework but different decision rules. First I create a list of potential matches using the time variable and then check for similarity using Sequence Matcher.

Both attempts resulted in a score of 93.75 on the contest page and thinking it to be sufficient I did not attempt a machine learning method.

In [7]:
def extractor_from_amazon(pd_df_row):
    # Input: pandas data frame iterrow item
    # Output: 
    
    #raw data
    am_id = pd_df_row['id'] 
    time = pd_df_row['time']
    director = pd_df_row['director']
    
   
    stars = str(pd_df_row['star']).split(', ')
    
        
    #start structuring
    
    #time as int and only minutes
    if type(time) is str and '/' not in time:
        
        if 'hour' in time:
            time = int(time.split()[0]) * 60 + int(time.split()[2])
            
        else:
            time = int(time.split()[0])
    else:
        time = int(99999)
        
    return am_id, director, time, stars
    

In [8]:
def extractor_from_rotten(pd_df_row):
    
    
    rot_id = pd_df_row['id']
    director = pd_df_row['director']
    time = pd_df_row['time']

   
    stars = []
    for star in pd_df_row[['star1','star2','star3','star4','star5','star6']]:
        if type(star) != float:
            stars.append(star)
        
        
    if type(time) is str:
        
        if 'hr' in time and 'min' in time:
            time = int(time.split()[0]) * 60 + int(time.split()[2])
            
        elif 'hr' in time:
            time = int(time.split()[0]) * 60
            
        elif 'min' in time:
            time = int(time.split()[0])
            
        else:
            time = int(99999)
    else:
        time = int(99999)
        
    return rot_id, director, time, stars
    
    

In [9]:
def make_new_df(amazoncsv, rottencsv):
    
    amazon = pd.read_csv(amazoncsv)
    rotten = pd.read_csv(rottencsv)
    
    returnamazon = pd.DataFrame(columns=['am_id','am_director','am_time','am_stars'],index=amazon.index.values)
    returnrotten = pd.DataFrame(columns=['rot_id','rot_director','rot_time','rot_stars'],index=rotten.index.values)

    for index, row in amazon.iterrows():
        returnamazon['am_id'].iloc[index], \
        returnamazon['am_director'].iloc[index], \
        returnamazon['am_time'].iloc[index], \
        returnamazon['am_stars'].iloc[index] = extractor_from_amazon(row)
    
    for index, row in rotten.iterrows():
        returnrotten['rot_id'].iloc[index], \
        returnrotten['rot_director'].iloc[index], \
        returnrotten['rot_time'].iloc[index], \
        returnrotten['rot_stars'].iloc[index] = extractor_from_rotten(row)
        
        
    return returnamazon, returnrotten
        
        
        

In [ ]:

#first model
def compare_id(amazon_id,rotten_id):
    
    row_amazon = amazon[amazon['am_id']==amazon_id] 
    row_rotten = rotten[rotten['rot_id']==rotten_id]
    
    am_id, am_director, am_time, am_stars = row_amazon.iloc[0,:]
    rot_id, rot_director, rot_time, rot_stars = row_rotten.iloc[0,:]

    # initial check that looks for all values to match
    if abs(am_time - rot_time) < 3 and am_director in rot_director and \
                                  False not in [i in rot_stars for i in am_stars]:
        return 1
        
         
    
    
    
    #relaxation of the first check, looks only for time and director 
    if  abs(am_time - rot_time) < 10 and SequenceMatcher(None, am_director, rot_director ).ratio()> 0.90:
        
        #in case of multiple potential matches checks how stars compare among the matches
        potential_matches = rotten[(rotten['rot_director']== am_director) & (abs(rotten['rot_time'] - am_time) < 10 )]
        if potential_matches.shape[0] > 1:  
            foo=[]
            for index,row in potential_matches.iterrows():
                foo.append(([row['rot_id'],([i in rot_stars for i in am_stars].count(True))/len(am_stars)]))
                
            #to do: check if multiple items with same length and return the one with closest time 
            if max(foo,key=itemgetter(1))[0] == rot_id:
                return 1
    
            else:
                return 0
        else: 
            return 1 
    
    
    
    else:
        return 0
    
    ]

In [39]:
# second model
def compare_id2(amazon_id,rotten_id,amazon,rotten):
    
    row_amazon = amazon[amazon['am_id']==amazon_id] 
    row_rotten = rotten[rotten['rot_id']==rotten_id]
    
    am_id, am_director, am_time, am_stars = row_amazon.iloc[0,:]
    rot_id, rot_director, rot_time, rot_stars = row_rotten.iloc[0,:]
    
    """
    if abs(am_time - rot_time) < 3 and am_director in rot_director and \
                                  False not in [i in rot_stars for i in am_stars]:
        return 1
    """
    # initial check that looks for all values to match
    potential_matches = rotten[ (abs(rotten['rot_time'] - am_time) < 15 )]
                           
    foo = [];percentage = 0
    for index, row in potential_matches.iterrows():

        ratio = ( 5 * SequenceMatcher(None, row['rot_director'],am_director).ratio() ) \
\
        + 0.1/ (1 + abs(row['rot_time'] - am_time )) \
\
        + np.sum([SequenceMatcher(None,i,j).ratio() \
                  for i in am_stars for j in rot_stars if SequenceMatcher(None,i,j).ratio() > 0.75])

    
        foo.append((row['rot_id'], ratio))

    if len(foo) > 1:
        if max(foo,key=itemgetter(1))[0] == rot_id:
            return 1
        

    
    return 0 
    

In [40]:
amazon, rotten = make_new_df('amazon.csv','rotten_tomatoes.csv')

In [41]:
def f1_score(goldtruth,goldpredict):

    falsepositive=0
    truepositive=0
    truenegative=0
    falsenegative=0
    
    index=0
    for truth in goldtruth:
        if truth == 0:
            if goldpredict.iloc[index,0] ==0:
                truenegative += 1
            else:
                falsepositive += 1
            
        if truth == 1:
            if goldpredict.iloc[index,0]==0:
                falsenegative += 1
                
            else:
                truepositive += 1
            
        index += 1
        
    precision = float(truepositive) / (truepositive + falsepositive)
    recall = float(truepositive) / (truepositive+falsenegative)
    
    print 'precision =', precision
    print 'recall =', recall
    return 2* precision * recall / (precision+recall)
    
    
    

In [35]:
def compare_multiple_ids(test_csv):
    start_time = time.time()
    
    test = pd.read_csv(test_csv)
    goldcsv = pd.DataFrame(columns=['gold'],index=test.index.values,dtype=int)

    
    amazon, rotten = make_new_df('amazon.csv','rotten_tomatoes.csv')
    
    counter = 1
    for index,row in test.iterrows():
        goldcsv['gold'][index] = compare_id2(row['id1'],row['id 2'],amazon,rotten)
        
        print counter
        counter+=1
        
        
    goldcsv.to_csv(test_csv + '_result.csv',index=False)

    print time.time()-start_time
    
    return goldcsv


In [42]:
goldpredict = compare_multiple_ids('holdout.csv')
goldpredict = compare_multiple_ids('test.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
209.370546818
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
188.966865063


In [97]:
goldtruth = pd.read_csv('tes.csv')['gold']

In [32]:

f1_score(goldtruth,goldpredict)

NameError: name 'goldtruth' is not defined